<a href="https://colab.research.google.com/github/Naii-the-Baf/colab/blob/main/Proy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import copy
import numpy as np
import pandas as pd
import random
import string

from sklearn.model_selection import train_test_split

In [ ]:
#Custom hash function for testing.
def XORHash(key : str):
  key_arr = bytearray()
  key_arr.extend(key.encode("UTF-8"))
  alnum_str = list(string.digits + "abcdef")
  output = []
  for i in range(len(key_arr)):
    byte_xor = key_arr[i] ^ key_arr[-i] ^ key_arr[i - 1]
    output.append(byte_xor)
  str_out = "".join((alnum_str[ch % len(alnum_str)] for ch in output))
  return str_out

In [ ]:
alnum_str = list(string.digits + "abcdef")
print(alnum_str)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f']


In [ ]:
#Collision in the hash function
print(XORHash("e3"), XORHash("ec"))

35 35


In [ ]:
def getRandomString (length = 1, alphabet = string.ascii_lowercase):
  return "".join(random.choice(alphabet) for i in range(length))

In [ ]:
#This creates (or overwrites) a file containing a hash table.
def writeDataset(filename = "./hashdb.csv", length = 1000):
  csvfile = open(filename, "w")
  alnum_str = list(string.digits + "abcdef")

  for i in range(length):
      input = getRandomString(5, alnum_str)
      csvfile.write(f"{XORHash(input)},{input}\n")

  csvfile.close()

In [ ]:
writeDataset(length=300000)

In [ ]:
#Converts a string into an array of numerical values.
def textToFeatures(string, alphabet):
  out_array = np.zeros(len(string), dtype=np.int32)
  for i in range(len(string)):
    out_array[i] = alphabet.index(string[i])
  return out_array

In [ ]:
#Converts an array of numerical values into a string.
def valuesToText(array, alphabet):
  return "".join((alphabet[char] for char in array))

In [ ]:
#Looks for a dataset and loads it. Returns a pandas DataFrame.
#The read file is expected to be a hash table with two columns.
def readDataset(filename = "./hashdb.csv"):
  return pd.read_csv(filename, header=None, dtype=str)

In [ ]:
dataset = readDataset()
string_len = len(dataset[0][0])

In [ ]:
#Remove duplicates
dataset = dataset[[0, 1]].drop_duplicates(subset=[0]).reset_index(drop=True)

In [ ]:
dataset

,0,1
0,52454,cde5e
1,8943e,deb38
2,8c74a,bf548
3,8b14e,ab748
4,8c76c,fb368
...,...,...
83836,9367c,8b379
83837,8fb3a,6a938
83838,4a003,9770d
83839,44904,0090d


In [ ]:
#Transforms every row to number and splits them into two dataframes.
def datasetTextToFeatures(df):
  x = pd.DataFrame(index=df.index,
                   columns=range(string_len),
                   dtype=int)
  y = pd.DataFrame(index=df.index,
                   columns=range(string_len),
                   dtype=int)
  for row in df.index:
      x.loc[row] = pd.Series(textToFeatures(df[0][row], alnum_str))
      y.loc[row] = pd.Series(textToFeatures(df[1][row], alnum_str))
  return x, y

In [ ]:
feat_data, feat_test = train_test_split(dataset, test_size = 0.2)

In [ ]:
data_x, data_y = datasetTextToFeatures(feat_data)

In [ ]:
test_x, test_y = datasetTextToFeatures(feat_test)

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers, Input

In [ ]:
#Loss function. Gets the reciprocal of the proportion of correct characters.
#Uses hamming distance.
def lossFunc(y_true, y_pred):
  """y_true_np = y_true.numpy()
  y_pred_np = y_pred.numpy()
  y_pred_np = y_pred_np.round()
  #print(y_true_np)
  #print(y_pred_np)

  cases = len(y_true_np)
  char_num = len(y_true_np[0])
  collected = []
  for i in range(cases):
    char_correct = 1
    for e in range(char_num):
      if y_true_np[i, e] == y_pred_np[i, e]:
        char_correct += 1
    collected.append(((char_num + 1) / char_correct) - 1)
  print(collected)
  return collected"""
  return tf.norm(y_true - y_pred, axis=-1)

In [82]:
model = models.Sequential([
    Input(shape=(5)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(5, activation='relu'),
])

In [83]:
model.compile(loss=lossFunc, optimizer='adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 64)                384       
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 5)                 325       
                                                                 
Total params: 4869 (19.02 KB)
Trainable params: 4869 (19.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [84]:
model.fit(x=data_x, y=data_y, epochs=300, validation_data=(test_x, test_y))

Epoch 1/300
2096/2096 [==============================] - 5s 2ms/step - loss: 11.4807 - val_loss: 9.1057
Epoch 2/300
2096/2096 [==============================] - 4s 2ms/step - loss: 9.0661 - val_loss: 8.9745
Epoch 3/300
2096/2096 [==============================] - 4s 2ms/step - loss: 8.9810 - val_loss: 8.9222
Epoch 4/300
2096/2096 [==============================] - 4s 2ms/step - loss: 8.9231 - val_loss: 8.8717
Epoch 5/300
2096/2096 [==============================] - 4s 2ms/step - loss: 8.8826 - val_loss: 8.8388
Epoch 6/300
2096/2096 [==============================] - 4s 2ms/step - loss: 8.8468 - val_loss: 8.8704
Epoch 7/300
2096/2096 [==============================] - 4s 2ms/step - loss: 8.8110 - val_loss: 8.7719
Epoch 8/300
2096/2096 [==============================] - 4s 2ms/step - loss: 8.7861 - val_loss: 8.7538
Epoch 9/300
2096/2096 [==============================] - 4s 2ms/step - loss: 8.7626 - val_loss: 8.8167
Epoch 10/300
2096/2096 [==============================] - 3s 2ms/step - 

In [85]:
rand_hash = pd.Series(textToFeatures(dataset[0][0], alnum_str))
pred = model.predict(x=[list(rand_hash)])
print(list(rand_hash), dataset[0][0])
print(np.round(pred), valuesToText(np.round(pred)[0].astype(int), alnum_str))

1/1 [==============================] - 0s 59ms/step
[5, 2, 4, 5, 4] 52454
[[ 6. 10.  7. 10.  9.]] 6a7a9
